# Introduction

The provided notebook is a Python program for cryptocurrency data analysis and portfolio optimization. It uses historical price data and market cap data for selected cryptocurrencies to perform portfolio optimization using the Efficient Frontier approach. The script aims to calculate the profit/loss of the portfolio over different time periods and with varying numbers of coins.

# Notebook
## Import libs

In [1]:
import sys
sys.path.append("../")

import itertools
from time import time
from datetime import datetime

import numpy as np
import pandas as pd


from src import cryptorama as cc

## Functions

In [2]:
def calculate_profit(df, _top_100, _n_coins, _mu_method, _cov_method, _obj_function, _budget, _n_days_vector, sell_day, _compounding,
                     _save_dir):

    crypto_class = cc.CryptoPortfolio(top_100, _budget, _n_coins, _save_dir)
    results = {}
    portf = {}

    for n_days in _n_days_vector:
        try:
            crypto_class.validate_from_past_specific_dates(_n_coins, float(n_days), sell_day, _mu_method, _cov_method, _obj_function, _compounding)
            date = df.iloc[n_days].name

            results[date] = crypto_class.p_l_specific
            portf[date] = crypto_class.portfolio_from_past_specific
        except Exception as e:
            pass

    p_l = sum(results.values())

    return p_l, results, portf

def check_coins(portfolio):
    # Convert the dictionary to a list of DataFrames
    dfs = [pd.DataFrame(portfolio[date]) for date in portfolio]

    # Concatenate the list of DataFrames into a single DataFrame
    result_df = pd.concat(dfs, keys=portfolio.keys())

    def get_total_coins(data):
        print(data["Coin"].iloc[0], data["n_coins"].sum())

    result_df.groupby("Coin").apply(get_total_coins)
    
    
def convert_date_to_number(date_latest_update, wanted_date):
    # Convert the date strings to datetime objects
    date_latest_update = datetime.strptime(date_latest_update, "%d/%m/%Y")
    wanted_date = datetime.strptime(wanted_date, "%d/%m/%Y")
    # date_sell = datetime.strptime(sell_date, "%d/%m/%Y")

    # Calculate the time delta
    delta_buy = date_latest_update - wanted_date
    # delta_sell = date_latest_update - date_sell

    # Convert the time delta to a float representing the number of days
    delta = int(delta_buy.total_seconds() / (24 * 60 * 60))
    # delta_sell = int(delta_sell.total_seconds() / (24 * 60 * 60))
    return delta    


## Inputs

In [3]:
start = time()

top_100 = False

# you place 100e each month per category for the past n years
budget = 100

save_dir = './legacy_data'

market_phase = "Bull"

most_recent = True

In [4]:
# Define the  dates
latest_update = "27/07/2023"

# Halving Dates
halving_2k16 = "09/07/2016"
halving_2k20 = "11/05/2020"

# Bear Dates 
bear_2k18 = "01/01/2018"
bear_2k22 = "01/01/2022"



market = dict()

market['Bull'] = {
    "2nd": (convert_date_to_number(latest_update, halving_2k16) + 365,
             convert_date_to_number(latest_update, halving_2k16) - 365),
    "3rd": (convert_date_to_number(latest_update, halving_2k20) + 365,
                           convert_date_to_number(latest_update, halving_2k20) - 365)}
market["Bear"] = {
    "2nd" : (convert_date_to_number(latest_update, bear_2k18), 
              convert_date_to_number(latest_update, halving_2k20) + 365),
    "3rd" : (convert_date_to_number(latest_update, bear_2k22), 0)}# Now

In [5]:
if most_recent:
    delta_buy = market[market_phase]["3rd"][0]
    delta_sell = market[market_phase]["3rd"][1]
else:
    delta_buy = market[market_phase]["2nd"][0]
    delta_sell = market[market_phase]["2nd"][1]      
    
    
print(f"\ndelta_buy : {delta_buy}")
print(f"\ndelta_sell : {delta_sell}")


delta_buy : 1537

delta_sell : 807


In [6]:
n_days_vector = np.arange(delta_buy, delta_sell, -30)
# df_mc = pd.read_csv('./legacy_data/All_cryptos_market_cap.csv', index_col=0)
df_mc = pd.read_csv(f'{save_dir}/All_cryptos_market_cap.csv', index_col=0)

## Optimization parameters

In [7]:
possible_mu_methods = ["mean", "exp", "capm"]
possible_cov_methods = ["sample", "exp"]
possible_obj_functions = ["quadratic", "sharpe", "min_volat"]
compounding_options = [True, False]

# Get all combinations of mu_method, cov_method, and obj_function
all_combinations = list(itertools.product(possible_mu_methods, possible_cov_methods, possible_obj_functions, compounding_options))

## Variables initialization

In [8]:
maxpl2 = 0
maxpl10 = 0
maxpl20 = 0


best_portf_2 = None
best_portf_10 = None
best_portf_20 = None

best_res_2 = None
best_res_10 = None 
best_res_20 = None  

optimized_variables = dict()

## Run Analysis

In [9]:
# Test the portfolio_optimization function with all combinations
start = time()
for mu_method, cov_method, obj_function, compounding in all_combinations:

    n_coins = 2
    print(f'\n===== {n_coins} COINS ======\n')
    
#     try:
    p_l_2, results_2, portf_2 = calculate_profit(df_mc, top_100, n_coins, mu_method, cov_method,
                                                obj_function, budget, n_days_vector, delta_sell, compounding, save_dir)
#     except Exception as e:
#         pass
    if p_l_2 >= maxpl2:
        maxpl2 = p_l_2
        best_portf_2 = portf_2
        best_res2 = results_2
        
        optimized_variables[f"{n_coins}_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function,
            "compounding": compounding}
        
    n_coins = 10
    print(f'\n===== {n_coins} COINS ======\n')
    
    try:
        p_l_10, results_10, portf_10 = calculate_profit(df_mc, top_100, n_coins, mu_method, cov_method,
                                                obj_function, budget, n_days_vector, delta_sell, compounding, save_dir)
    except Exception as e:
        pass
    if p_l_10 >= maxpl10:
        maxpl10 = p_l_10
        best_portf_10 = portf_10
        best_res10 = results_10
        
        optimized_variables[f"{n_coins}_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function,
            "compounding": compounding}
            
    n_coins = 20

    print(f'\n===== {n_coins} COINS ======\n')

    try:
        p_l_20, results_20, portf_20 = calculate_profit(df_mc, top_100, n_coins, mu_method, cov_method,
                                                obj_function, budget, n_days_vector, delta_sell, compounding, save_dir)
    except Exception as e:
        pass
    
    if p_l_20 >= maxpl20:
        maxpl20 = p_l_20
        best_portf_20 = portf_20
        best_res20 = results_20
        
        optimized_variables[f"{n_coins}_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function,
            "compounding": compounding}
        
        

end = time()
print(f'Time elapsed: {(end - start) / 60}')


===== 2 COINS ======

Expected annual return: 255.8%
Annual volatility: 123.2%
Sharpe Ratio: 2.06
Invest 100.0

 Profit Loss: 2125.380417974375e

Expected annual return: 255.8%
Annual volatility: 123.2%
Sharpe Ratio: 2.06
Invest 100.0

 Profit Loss: 1596.098155437815e

Expected annual return: 255.8%
Annual volatility: 123.2%
Sharpe Ratio: 2.06
Invest 100.0

 Profit Loss: 1451.4098894943131e

Expected annual return: 255.8%
Annual volatility: 123.2%
Sharpe Ratio: 2.06
Invest 100.0

 Profit Loss: 1916.4609785860546e

Expected annual return: 255.8%
Annual volatility: 123.2%
Sharpe Ratio: 2.06
Invest 100.0

 Profit Loss: 2201.2494084595296e

Expected annual return: 255.8%
Annual volatility: 123.2%
Sharpe Ratio: 2.06
Invest 100.0

 Profit Loss: 2056.6717293301235e

Expected annual return: 255.8%
Annual volatility: 123.2%
Sharpe Ratio: 2.06
Invest 100.0

 Profit Loss: 2162.9975440216986e

Expected annual return: 255.8%
Annual volatility: 123.2%
Sharpe Ratio: 2.06
Invest 100.0

 Profit Loss: 

Expected annual return: 3488.5%
Annual volatility: 155.6%
Sharpe Ratio: 22.41
Invest 100.0

 Profit Loss: 572.2890128009477e

Expected annual return: 3488.5%
Annual volatility: 155.6%
Sharpe Ratio: 22.41
Invest 100.0

 Profit Loss: 849.4310117790866e

Expected annual return: 3488.5%
Annual volatility: 155.6%
Sharpe Ratio: 22.41
Invest 100.0

 Profit Loss: 858.810514482853e

Expected annual return: 3488.5%
Annual volatility: 155.6%
Sharpe Ratio: 22.41
Invest 100.0

 Profit Loss: 607.1956954590017e

Expected annual return: 3488.5%
Annual volatility: 155.6%
Sharpe Ratio: 22.41
Invest 100.0

 Profit Loss: 365.18751428291984e

Expected annual return: 7364393.0%
Annual volatility: 13093.9%
Sharpe Ratio: 562.43
Invest 100.0

 Profit Loss: 52.36091512427068e

Expected annual return: 7364393.0%
Annual volatility: 13093.9%
Sharpe Ratio: 562.43
Invest 100.0

 Profit Loss: 16.947392601203063e

Expected annual return: 4031.2%
Annual volatility: 203.9%
Sharpe Ratio: 19.76
Invest 100.0

 Profit Loss:

Expected annual return: 307.5%
Annual volatility: 111.0%
Sharpe Ratio: 2.75
Invest 99.99999999999999

 Profit Loss: 1825.9297611116629e

Expected annual return: 307.5%
Annual volatility: 111.0%
Sharpe Ratio: 2.75
Invest 99.99999999999999

 Profit Loss: 2000.4506947339985e

Expected annual return: 314.6%
Annual volatility: 133.1%
Sharpe Ratio: 2.35
Invest 100.0

 Profit Loss: 2495.6748264548164e

Expected annual return: 307.5%
Annual volatility: 111.0%
Sharpe Ratio: 2.75
Invest 99.99999999999999

 Profit Loss: 3115.731484586173e

Expected annual return: 307.5%
Annual volatility: 111.0%
Sharpe Ratio: 2.75
Invest 99.99999999999999

 Profit Loss: 2667.723336926091e

Expected annual return: 307.5%
Annual volatility: 111.0%
Sharpe Ratio: 2.75
Invest 99.99999999999999

 Profit Loss: 3410.4075717613323e

Expected annual return: 307.5%
Annual volatility: 111.0%
Sharpe Ratio: 2.75
Invest 99.99999999999999

 Profit Loss: 3633.5737101435207e

Expected annual return: 307.5%
Annual volatility: 111.0

Expected annual return: 903.4%
Annual volatility: 151.6%
Sharpe Ratio: 5.94
Invest 100.0

 Profit Loss: 2917.306584504489e

Expected annual return: 3055.3%
Annual volatility: 134.6%
Sharpe Ratio: 22.68
Invest 100.0

 Profit Loss: 520.5879564285668e

Expected annual return: 3055.3%
Annual volatility: 134.6%
Sharpe Ratio: 22.68
Invest 100.0

 Profit Loss: 785.3941990736813e

Expected annual return: 3055.3%
Annual volatility: 134.6%
Sharpe Ratio: 22.68
Invest 100.0

 Profit Loss: 786.5734890321819e

Expected annual return: 3055.3%
Annual volatility: 134.6%
Sharpe Ratio: 22.68
Invest 100.0

 Profit Loss: 561.4268500997271e

Expected annual return: 3055.3%
Annual volatility: 134.6%
Sharpe Ratio: 22.68
Invest 100.0

 Profit Loss: 354.8307006139455e

Expected annual return: 3055.3%
Annual volatility: 134.6%
Sharpe Ratio: 22.68
Invest 100.0

 Profit Loss: 138.91127038519264e

Expected annual return: 3055.3%
Annual volatility: 134.6%
Sharpe Ratio: 22.68
Invest 100.0

 Profit Loss: 19.2082825187

Expected annual return: 267.6%
Annual volatility: 109.6%
Sharpe Ratio: 2.42
Invest 100.0

 Profit Loss: 2547.7561367843746e

Expected annual return: 267.6%
Annual volatility: 109.6%
Sharpe Ratio: 2.42
Invest 100.0

 Profit Loss: 2376.3564230691472e

Expected annual return: 267.6%
Annual volatility: 109.6%
Sharpe Ratio: 2.42
Invest 100.0

 Profit Loss: 2987.5671955486496e

Expected annual return: 264.4%
Annual volatility: 108.4%
Sharpe Ratio: 2.42
Invest 99.99999999999999

 Profit Loss: 3037.340702283286e

Expected annual return: 271.1%
Annual volatility: 110.7%
Sharpe Ratio: 2.43
Invest 100.0

 Profit Loss: 2382.211242063718e

Expected annual return: 270.4%
Annual volatility: 111.2%
Sharpe Ratio: 2.41
Invest 100.00000000000001

 Profit Loss: 2228.3453297227798e

Expected annual return: 270.4%
Annual volatility: 111.2%
Sharpe Ratio: 2.41
Invest 100.00000000000001

 Profit Loss: 3040.887312288476e

Expected annual return: 270.4%
Annual volatility: 111.2%
Sharpe Ratio: 2.41
Invest 100.000

...

Expected annual return: 245.8%
Annual volatility: 92.9%
Sharpe Ratio: 2.62
Invest 100.0

 Profit Loss: 1364.8505423930271e

Expected annual return: 245.8%
Annual volatility: 92.9%
Sharpe Ratio: 2.62
Invest 100.0

 Profit Loss: 911.9410118195776e

Expected annual return: 230.4%
Annual volatility: 79.2%
Sharpe Ratio: 2.88
Invest 100.0

 Profit Loss: 804.2028882924815e

Expected annual return: 230.4%
Annual volatility: 79.2%
Sharpe Ratio: 2.88
Invest 100.00000000000001

 Profit Loss: 805.7838262975253e

Expected annual return: 236.2%
Annual volatility: 80.5%
Sharpe Ratio: 2.91
Invest 100.0

 Profit Loss: 688.957178192707e

Expected annual return: 248.9%
Annual volatility: 79.9%
Sharpe Ratio: 3.09
Invest 100.0

 Profit Loss: 597.5824331743704e

Expected annual return: 248.9%
Annual volatility: 79.9%
Sharpe Ratio: 3.09
Invest 99.99999999999999

 Profit Loss: 482.35760956050314e

Expected annual return: 248.9%
Annual volatility: 79.9%
Sharpe Ratio: 3.09
Invest 99.99999999999999

 Profit Loss

Expected annual return: 154.8%
Annual volatility: 87.1%
Sharpe Ratio: 1.75
Invest 100.0

 Profit Loss: 706.9983498851074e

Expected annual return: 154.8%
Annual volatility: 87.1%
Sharpe Ratio: 1.75
Invest 100.0

 Profit Loss: 860.8693803415408e

Expected annual return: 153.2%
Annual volatility: 92.4%
Sharpe Ratio: 1.64
Invest 100.0

 Profit Loss: 1080.3763100200104e

Expected annual return: 153.2%
Annual volatility: 92.4%
Sharpe Ratio: 1.64
Invest 100.0

 Profit Loss: 1029.3208103369725e

Expected annual return: 153.2%
Annual volatility: 92.4%
Sharpe Ratio: 1.64
Invest 100.0

 Profit Loss: 1310.6123288948554e

Expected annual return: 154.8%
Annual volatility: 87.1%
Sharpe Ratio: 1.75
Invest 100.0

 Profit Loss: 1189.8231957565665e

Expected annual return: 169.0%
Annual volatility: 93.1%
Sharpe Ratio: 1.79
Invest 100.0

 Profit Loss: 1453.6113419078602e

Expected annual return: 155.2%
Annual volatility: 91.9%
Sharpe Ratio: 1.67
Invest 100.0

 Profit Loss: 985.277680024139e

Expected ann

Expected annual return: 144.6%
Annual volatility: 75.2%
Sharpe Ratio: 1.90
Invest 100.0

 Profit Loss: 527.7192656822726e

Expected annual return: 144.6%
Annual volatility: 75.2%
Sharpe Ratio: 1.90
Invest 100.0

 Profit Loss: 409.21551270770067e

Expected annual return: 144.6%
Annual volatility: 75.2%
Sharpe Ratio: 1.90
Invest 100.0

 Profit Loss: 249.72219625668566e

Expected annual return: 144.6%
Annual volatility: 75.2%
Sharpe Ratio: 1.90
Invest 100.0

 Profit Loss: 144.23928175899954e

Expected annual return: 144.6%
Annual volatility: 75.2%
Sharpe Ratio: 1.90
Invest 100.0

 Profit Loss: 90.99713432161795e

Expected annual return: 144.6%
Annual volatility: 75.2%
Sharpe Ratio: 1.90
Invest 100.0

 Profit Loss: 39.177483586921376e

Expected annual return: 144.6%
Annual volatility: 75.2%
Sharpe Ratio: 1.90
Invest 100.0

 Profit Loss: 11.5240294792171e

Expected annual return: 144.6%
Annual volatility: 75.2%
Sharpe Ratio: 1.90
Invest 100.0

 Profit Loss: 3.9466552187628103e


===== 10 CO

Expected annual return: 118.4%
Annual volatility: 75.2%
Sharpe Ratio: 1.55
Invest 100.0

 Profit Loss: 921.559184880143e

Expected annual return: 118.4%
Annual volatility: 75.2%
Sharpe Ratio: 1.55
Invest 100.0

 Profit Loss: 894.0747998314046e

Expected annual return: 118.4%
Annual volatility: 75.2%
Sharpe Ratio: 1.55
Invest 100.0

 Profit Loss: 669.8969141906782e

Expected annual return: 118.4%
Annual volatility: 75.2%
Sharpe Ratio: 1.55
Invest 100.0

 Profit Loss: 687.7105412048487e

Expected annual return: 118.4%
Annual volatility: 75.2%
Sharpe Ratio: 1.55
Invest 100.0

 Profit Loss: 908.1628299103425e

Expected annual return: 118.4%
Annual volatility: 75.2%
Sharpe Ratio: 1.55
Invest 100.0

 Profit Loss: 705.7583347214297e

Expected annual return: 118.4%
Annual volatility: 75.2%
Sharpe Ratio: 1.55
Invest 100.0

 Profit Loss: 641.3856409892994e

Expected annual return: 118.4%
Annual volatility: 75.2%
Sharpe Ratio: 1.55
Invest 100.0

 Profit Loss: 688.3987823379273e

Expected annual r

Expected annual return: 62.5%
Annual volatility: 46.9%
Sharpe Ratio: 1.29
Invest 100.0

 Profit Loss: 12.329922346156806e

Expected annual return: 63.3%
Annual volatility: 46.0%
Sharpe Ratio: 1.33
Invest 100.0

 Profit Loss: 3.604917387759769e

Time elapsed: 12.067213253180187


In [10]:
check_coins(best_portf_2)
print(maxpl2)
optimized_variables["2_coins"]

ETH 9.18997475780779
35810.25740994037


{'mu': 'capm', 's': 'exp', 'obj_function': 'quadratic', 'compounding': False}

In [11]:
check_coins(best_portf_20)
print(maxpl20)
optimized_variables["20_coins"]

BNB 75.47517606422545
ETC 75.01287416318752
LUNC 0.9350581616869738
SOL 2.019874704198843
UNI 2.8770923246912146
56997.50677363453


{'mu': 'exp', 's': 'sample', 'obj_function': 'quadratic', 'compounding': True}

In [12]:
check_coins(best_portf_10)
print(maxpl10)
optimized_variables["10_coins"]

BNB 102.5078891897537
UNI 5.967116012452854
66649.20780149243


{'mu': 'exp', 's': 'exp', 'obj_function': 'quadratic', 'compounding': True}

## Results

In [13]:
print(f'Profit 2c: {round(maxpl2, 2)} $')
print(f'Profit 10c: {round(maxpl10, 2)} $')
print(f'Profit 20c: {round(maxpl20, 2)} $')
# print(f'Profit 100c: {round(maxpl100, 2)} $')

print(f'\nInvestment: {3 * budget * len(n_days_vector)} $')
print(f'Final Profit: {round(maxpl2 + maxpl10 + maxpl20 , 2)} $') # + maxpl100
print(f'Final Profit: {round((maxpl2 + maxpl10 + maxpl20 ) / (3 * budget * len(n_days_vector)) * 100, 2)} %\n') # + maxpl100

end = time()
print(f'Time elapsed: {(end - start) / 60}')

Profit 2c: 35810.26 $
Profit 10c: 66649.21 $
Profit 20c: 56997.51 $

Investment: 7500 $
Final Profit: 159456.97 $
Final Profit: 2126.09 %

Time elapsed: 12.068217519919077


# Conclusion

## 2 Coins

In [14]:
optimized_variables["2_coins"]

{'mu': 'capm', 's': 'exp', 'obj_function': 'quadratic', 'compounding': False}

## 10 Coins

In [15]:
optimized_variables["10_coins"]

{'mu': 'exp', 's': 'exp', 'obj_function': 'quadratic', 'compounding': True}

## 20 Coins

In [16]:
optimized_variables["20_coins"]

{'mu': 'exp', 's': 'sample', 'obj_function': 'quadratic', 'compounding': True}

# Results

## Bear Market 2022

### 2 Coins

{'mu': 'capm',
 's': 'sample',
 'obj_function': 'min_volat',
 'compounding': False}
 
### 10 Coins 

{'mu': 'exp', 's': 'sample', 'obj_function': 'quadratic', 'compounding': True}

### 20 Coins

{'mu': 'exp', 's': 'sample', 'obj_function': 'quadratic', 'compounding': True}

## Bear Market 2018

### 2 Coins:

{'mu': 'exp', 's': 'exp', 'obj_function': 'sharpe', 'compounding': False}

### 10 Coins 

{'mu': 'exp', 's': 'sample', 'obj_function': 'sharpe', 'compounding': False}

### 20 Coins

{'mu': 'mean', 's': 'exp', 'obj_function': 'quadratic', 'compounding': True}

If i had applied these parameters in the bear market of 2022 i would have a profit of ~6%

## Bull Market 2016

### 2 Coins:

{'mu': 'exp', 's': 'exp', 'obj_function': 'sharpe', 'compounding': True}


### 10 Coins 

{'mu': 'exp', 's': 'exp', 'obj_function': 'quadratic', 'compounding': True}

### 20 Coins

{'mu': 'exp', 's': 'exp', 'obj_function': 'quadratic', 'compounding': True}


If i had applied these parameters in the bear market of 2022 i would have a profit of ~2106%

## Bull Market 2020

### 2 Coins:

{'mu': 'capm', 's': 'exp', 'obj_function': 'quadratic', 'compounding': False}


### 10 Coins 

{'mu': 'exp', 's': 'exp', 'obj_function': 'quadratic', 'compounding': True}

### 20 Coins

{'mu': 'exp', 's': 'sample', 'obj_function': 'quadratic', 'compounding': True}
